In [13]:
import pandas as pd 

In [14]:
file=pd.read_csv(r"C:\Users\ASUS\My_Learning\sat hack\event_log_india.csv")
file.head()

,event_id,stream_name,event_type,payload_json,published_at,processed
0,E000001,predictions,trip.completed,"{""sample"": 0}",2025-11-13T10:05:24.782600,False
1,E000002,predictions,notification.sent,"{""sample"": 1}",2025-11-13T06:05:24.782631,True
2,E000003,notifications,trip.completed,"{""sample"": 2}",2025-11-13T03:16:24.782649,False
3,E000004,predictions,prediction.made,"{""sample"": 3}",2025-11-13T04:52:24.782666,True
4,E000005,predictions,prediction.made,"{""sample"": 4}",2025-11-13T02:56:24.782676,False


In [26]:
import pandas as pd
import numpy as np

class RewardAgent:
    def __init__(self, trips, assignments, drivers, vehicles, fuel_preds=None, co2_data=None, events=None):
        self.trips = trips.copy()
        self.assignments = assignments.copy()
        self.drivers = drivers.copy()
        self.vehicles = vehicles.copy()
        self.fuel_preds = fuel_preds.copy() if fuel_preds is not None else None
        self.co2_data = co2_data.copy() if co2_data is not None else None
        self.events = events.copy() if events is not None else None

        # Live segments storage
        self.live_segments = pd.DataFrame(columns=['assignment_id','fuel_used_l','co2_kg'])

        # Strip column names
        for df in [self.trips, self.assignments, self.drivers, self.vehicles, self.fuel_preds, self.co2_data, self.events]:
            if df is not None:
                df.columns = df.columns.str.strip().str.lower()

        # Merge all data into reward_data
        self.reward_data = self._merge_data()

        # Initial reward computation
        self._compute_rewards()

    # ---------------- Merge all relevant data ----------------
    def _merge_data(self):
        df = self.trips.merge(self.assignments, on='assignment_id', how='left', suffixes=('_trip','_assign'))

        # Merge drivers using trips driver_id column
        df = df.merge(self.drivers, left_on='driver_id_trip', right_on='driver_id', how='left', suffixes=('','_driver'))

        # Merge vehicles using trip vehicle_id
        df = df.merge(self.vehicles, on='vehicle_id', how='left', suffixes=('','_vehicle'))

        # Merge fuel predictions
        if self.fuel_preds is not None:
            df = df.merge(
                self.fuel_preds[['assignment_id','predicted_fuel_l','predicted_co2_kg']],
                on='assignment_id', how='left'
            )
            df.rename(columns={'predicted_fuel_l':'predicted_fuel_l_pred',
                               'predicted_co2_kg':'predicted_co2_kg_pred'}, inplace=True)
        else:
            df['predicted_fuel_l_pred'] = np.nan
            df['predicted_co2_kg_pred'] = np.nan

        # Merge actual CO2 if available
        if self.co2_data is not None:
            df = df.merge(self.co2_data[['assignment_id','co2_kg','fuel_used_l']], on='assignment_id', how='left')
        else:
            df['co2_kg'] = np.nan
            df['fuel_used_l'] = np.nan

        # Event counts
        if self.events is not None:
            event_count = self.events.groupby('assignment_id').size().rename('event_count')
            df = df.merge(event_count, on='assignment_id', how='left')
        else:
            df['event_count'] = 0

        # Fill missing columns safely
        df['anomaly_flag'] = df.get('anomaly_flag', False)
        df['eco_score'] = df.get('eco_score', 0)
        df['reliability_rating'] = df.get('reliability_rating', 0)
        df['trips_completed'] = df.get('trips_completed', 0)
        df['scheduled_start'] = df.get('scheduled_start')
        df['scheduled_end'] = df.get('scheduled_end')
        df['start_time'] = df.get('start_time')
        df['end_time'] = df.get('end_time')

        return df

    # ---------------- Reward Functions ----------------
    def _fuel_reward(self, row):
        if pd.notna(row.get('predicted_fuel_l_pred')) and pd.notna(row.get('fuel_used_l')):
            return max(0, row['predicted_fuel_l_pred'] - row['fuel_used_l'])
        return 0

    def _co2_reward(self, row):
        if pd.notna(row.get('predicted_co2_kg_pred')) and pd.notna(row.get('co2_kg')):
            return max(0, row['predicted_co2_kg_pred'] - row['co2_kg'])
        return 0

    def _time_reward(self, row):
        if pd.notna(row.get('scheduled_start')) and pd.notna(row.get('scheduled_end')) \
           and pd.notna(row.get('start_time')) and pd.notna(row.get('end_time')):
            scheduled = row['scheduled_end'] - row['scheduled_start']
            actual = row['end_time'] - row['start_time']
            return (scheduled.total_seconds()/60) - (actual.total_seconds()/60)
        return 0

    def _driver_reward(self, row):
        eco_score = row.get('eco_score', 0)
        reliability = row.get('reliability_rating', 0)
        trips_completed = row.get('trips_completed', 0)
        return eco_score*0.5 + reliability*0.3 + trips_completed*0.1

    def _event_penalty(self, row):
        return row.get('event_count', 0) * 5

    def _anomaly_penalty(self, row):
        return 10 if row.get('anomaly_flag', False) else 0

    # ---------------- Compute rewards including live segments ----------------
    def _compute_rewards(self):
        df = self.reward_data.copy()

        # Apply live segment updates if any
        if not self.live_segments.empty:
            live_agg = self.live_segments.groupby('assignment_id')[['fuel_used_l','co2_kg']].sum().reset_index()
            live_agg.rename(columns={'fuel_used_l':'fuel_used_l_live', 'co2_kg':'co2_kg_live'}, inplace=True)
            df = df.merge(live_agg, on='assignment_id', how='left')

            # Combine live with existing
            df['fuel_used_l'] = df.get('fuel_used_l_live').combine_first(df.get('fuel_used_l'))
            df['co2_kg'] = df.get('co2_kg_live').combine_first(df.get('co2_kg'))

            df.drop(columns=['fuel_used_l_live','co2_kg_live'], inplace=True, errors='ignore')

        # Compute reward columns
        df['fuel_reward'] = df.apply(self._fuel_reward, axis=1)
        df['co2_reward'] = df.apply(self._co2_reward, axis=1)
        df['time_reward'] = df.apply(self._time_reward, axis=1)
        df['driver_reward'] = df.apply(self._driver_reward, axis=1)
        df['event_penalty'] = df.apply(self._event_penalty, axis=1)
        df['anomaly_penalty'] = df.apply(self._anomaly_penalty, axis=1)

        # Weighted total reward
        df['total_reward'] = (
            0.3*df['fuel_reward'] +
            0.3*df['co2_reward'] +
            0.2*df['time_reward'] +
            0.1*df['driver_reward'] -
            0.05*df['event_penalty'] -
            0.05*df['anomaly_penalty']
        )

        self.reward_data = df

    # ---------------- Public Method ----------------
    def calculate_rewards(self):
        self._compute_rewards()
        return self.reward_data

    # ---------------- Live Segment Update ----------------
    def update_segment(self, assignment_id, fuel_used_l=None, co2_kg=None):
        segment = {'assignment_id': assignment_id,
                   'fuel_used_l': fuel_used_l if fuel_used_l is not None else 0,
                   'co2_kg': co2_kg if co2_kg is not None else 0}
        self.live_segments = pd.concat([self.live_segments, pd.DataFrame([segment])], ignore_index=True)
        print(f"Updated live segment for assignment {assignment_id}")
        return self.calculate_rewards()


# ================================
# DEMO
# ================================
if __name__ == "__main__":
    trips = pd.DataFrame([
        {'assignment_id':'A1','trip_id':'T1','vehicle_id':'V1','driver_id_trip':'D1'},
        {'assignment_id':'A2','trip_id':'T2','vehicle_id':'V2','driver_id_trip':'D2'}
    ])
    assignments = pd.DataFrame([
        {'assignment_id':'A1'},
        {'assignment_id':'A2'}
    ])
    drivers = pd.DataFrame([
        {'driver_id':'D1','eco_score':80,'reliability_rating':9,'trips_completed':15},
        {'driver_id':'D2','eco_score':70,'reliability_rating':8,'trips_completed':12}
    ])
    vehicles = pd.DataFrame([
        {'vehicle_id':'V1'},
        {'vehicle_id':'V2'}
    ])
    fuel_preds = pd.DataFrame([
        {'assignment_id':'A1','predicted_fuel_l':10,'predicted_co2_kg':25},
        {'assignment_id':'A2','predicted_fuel_l':15,'predicted_co2_kg':30}
    ])
    co2_data = pd.DataFrame([
        {'assignment_id':'A1','fuel_used_l':9,'co2_kg':22},
        {'assignment_id':'A2','fuel_used_l':14,'co2_kg':28}
    ])
    events = pd.DataFrame([
        {'assignment_id':'A1','event_type':'minor'},
        {'assignment_id':'A2','event_type':'major'}
    ])

    agent = RewardAgent(trips, assignments, drivers, vehicles, fuel_preds, co2_data, events)

    print("Initial Rewards:")
    print(agent.calculate_rewards()[['trip_id','assignment_id','total_reward']])

    # Update a live segment
    agent.update_segment('A2', fuel_used_l=11, co2_kg=27)

    print("\nRewards after live segment update:")
    print(agent.calculate_rewards()[['trip_id','assignment_id','total_reward']])


Initial Rewards:
  trip_id assignment_id  total_reward
0      T1            A1          5.37
1      T2            A2          4.51
Updated live segment for assignment A2

Rewards after live segment update:
  trip_id assignment_id  total_reward
0      T1            A1          5.37
1      T2            A2          5.71
